<a href="https://colab.research.google.com/github/dohyeon-kim012/MachineLearning-DeepLearning/blob/main/DeepLearning/%EC%8B%A4%EC%8A%B5_CNN%20in%20class.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf

class MyModel(tf.keras.Model):

    def __init__(self):
        super(MyModel, self).__init__()

        self.conv1 = tf.keras.layers.Conv2D(32, 3, padding='SAME', input_shape=(28, 28, 1), activation='relu')
        self.conv2 = tf.keras.layers.Conv2D(32, 3, padding='SAME', activation='relu')
        self.pool1 = tf.keras.layers.MaxPool2D(2)
        self.drop1 = tf.keras.layers.Dropout(0.25)

        self.conv3 = tf.keras.layers.Conv2D(64, 3, padding='SAME', activation='relu')
        self.conv4 = tf.keras.layers.Conv2D(64, 3, padding='SAME', activation='relu')
        self.pool2 = tf.keras.layers.MaxPool2D(2)
        self.drop2 = tf.keras.layers.Dropout(0.25)

        self.flatten = tf.keras.layers.Flatten()
        self.dense1 = tf.keras.layers.Dense(512, activation='relu')
        self.dense2 = tf.keras.layers.Dense(10, activation='softmax')

    def call(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.pool1(x)
        x = self.drop1(x)

        x = self.conv3(x)
        x = self.conv4(x)
        x = self.pool2(x)
        x = self.drop2(x)

        x = self.flatten(x)
        x = self.dense1(x)
        out = self.dense2(x)
        
        return out

In [ ]:
@tf.function
def train_step(model, images, labels, loss_object, optimizer, train_loss, train_accuracy):

    with tf.GradientTape() as tape:
        prediction = model(images)
        loss = loss_object(labels, prediction)

    grads = tape.gradient(loss, model.trainable_variables)

    optimizer.apply_gradients(zip(grads, model.trainable_variables))

    train_loss(loss)
    train_accuracy(labels, prediction)

@tf.function
def test_step(model, images, labels, loss_object, test_loss, test_accuracy):
    prediction = model(images)

    t_loss = loss_object(labels, prediction)

    test_loss(t_loss)
    test_accuracy(labels, prediction)

In [ ]:
mnist = tf.keras.datasets.mnist

(X_train, y_train), (X_test, y_test) = mnist.load_data()

# 픽셀 정규화
X_train, X_test = X_train / 255., X_test / 255.

# 이미지 채널 차원 추가
X_train = X_train[..., tf.newaxis]
X_test = X_test[..., tf.newaxis]

# Tensorflow Dataset 만들기
train_ds = tf.data.Dataset.from_tensor_slices((X_train, y_train)).shuffle(1024).batch(32)
test_ds = tf.data.Dataset.from_tensor_slices((X_test, y_test)).batch(32)

11493376/11490434 [==============================] - 0s 0us/step


In [ ]:
model = MyModel()

In [ ]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam()

In [ ]:
train_loss = tf.keras.metrics.Mean(name="train_loss")
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name="test_accuracy")

In [ ]:
EPOCHS = 10

for epoch in range(EPOCHS):

    # 훈련
    for images, labels in train_ds:
        train_step(model, images, labels, loss_object, optimizer, train_loss, train_accuracy)

    # 테스트
    for test_images, test_labels in test_ds:
        test_step(model, images, labels, loss_object, test_loss, test_accuracy)

    template = "Epoch {}, Train Loss : {:.3f}, Train Accuracy : {:.3f}, Test Loss : {:.3f}, Test Accuracy : {:.3f}"
    print(template.format(epoch + 1,
                          train_loss.result(),
                          train_accuracy.result() * 100,
                          
                          test_loss.result(),
                          test_accuracy.result() * 100))

Epoch 1, Train Loss : 0.096, Train Accuracy : 97.043, Test Loss : 0.002, Test Accuracy : 100.000
Epoch 2, Train Loss : 0.065, Train Accuracy : 97.994, Test Loss : 0.002, Test Accuracy : 100.000
Epoch 3, Train Loss : 0.051, Train Accuracy : 98.404, Test Loss : 0.001, Test Accuracy : 100.000
Epoch 4, Train Loss : 0.043, Train Accuracy : 98.663, Test Loss : 0.006, Test Accuracy : 100.000
Epoch 5, Train Loss : 0.037, Train Accuracy : 98.838, Test Loss : 0.009, Test Accuracy : 100.000
Epoch 6, Train Loss : 0.033, Train Accuracy : 98.969, Test Loss : 0.008, Test Accuracy : 100.000
Epoch 7, Train Loss : 0.030, Train Accuracy : 99.073, Test Loss : 0.007, Test Accuracy : 100.000
Epoch 8, Train Loss : 0.027, Train Accuracy : 99.154, Test Loss : 0.006, Test Accuracy : 100.000
Epoch 9, Train Loss : 0.025, Train Accuracy : 99.216, Test Loss : 0.005, Test Accuracy : 100.000
Epoch 10, Train Loss : 0.023, Train Accuracy : 99.272, Test Loss : 0.005, Test Accuracy : 100.000
